In [ ]:
# Initialize the environment
import time, os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Prepare data 
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Check data
print train_images.shape
print len(train_labels)
print test_images.shape
print len(test_labels)

In [ ]:
#  Preprocess data
#plt.figure()
#plt.imshow(train_images[0])
#plt.colorbar()
#plt.gca().grid(False)

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Create an simple model
def createModel(lr,reg):
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(28, 28)),
            keras.layers.Dense(128, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(reg)),
            keras.layers.Dense(10, activation=tf.nn.softmax, kernel_regularizer=regularizers.l2(reg))
    ])
    model.compile(optimizer=tf.train.AdamOptimizer(lr), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# Create tool for visualizeing quick scan
def visQuickScan(results):
    import math
    x_scatter = [math.log10(x[0]) for x in results]
    y_scatter = [math.log10(x[1]) for x in results]

    # plot training accuracy
    marker_size = 100
    colors = [results[x][0] for x in results]
    plt.subplot(2, 1, 1)
    plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
    plt.colorbar()
    plt.xlabel('log learning rate')
    plt.ylabel('log regularization')
    plt.title('training accuracy')

    # plot test accuracy
    colors = [results[x][1] for x in results] # default size of markers is 20
    plt.subplot(2, 1, 2)
    plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
    plt.colorbar()
    plt.xlabel('log learning rate')
    plt.ylabel('log regularization')
    plt.title(' test accuracy')
    plt.show()

In [ ]:
# Create quick scan loop to find out the entry point
def quickScan(lr_range=[-3.0,-4.0],reg_range=[1,0],epoch=5,batch_size=10,sample=10):
    # init
    results = {}
    learning_rates = lr_range
    regularization_strengths = reg_range
    best_val = -1
    best_lr, best_reg = 0, 0
    
    # Timing
    tic = time.time()
    for i in range(sample):
        lr = 10**np.random.uniform(learning_rates[0],learning_rates[1])
        reg = 10**np.random.uniform(regularization_strengths[0],regularization_strengths[1])
        print 'No.', str(i)
        print 'lr:', str(lr)
        print 'reg:', str(reg)
        
        # model
        model = createModel(lr,reg)
        #model.summary()
        
        # training
        model.fit(train_images, train_labels,
                  validation_data = (test_images,test_labels),
                  epochs=epoch,
                  batch_size=batch_size,
                  shuffle=True)
        
        # evaluate result
        train_loss, train_acc = model.evaluate(train_images, train_labels)
        test_loss, test_acc = model.evaluate(test_images, test_labels)
        print('Train accuracy:', train_acc)
        print('Test accuracy:', test_acc)
        
        # record
        results[(lr,reg)] = (train_acc,test_acc)
        if best_val < test_acc:
            print 'Found Better model!!!'
            best_val = test_acc
            best_lr = lr
            best_reg = reg
            model.save_weights('./weight/myBestweight')
    
    toc = time.time()
    print 'Total Training: computed in %fs' % (toc - tic)
    print 'Best Validation Record %.5f' % (best_val)
    print 'Best Validation learning rate %.10f' % (best_lr)
    print 'Best Validation regularization %.10f' % (best_reg)
    
    # Visaulize the distribution
    visQuickScan(results)
    
    return best_lr, best_reg
  

In [ ]:
# Create the quick scan loop with the random learning rate & L2
best_lr, best_reg = quickScan(lr_range=[-3.0,-4.0],reg_range=[-4.0,-5.0],epoch=5,batch_size=10,sample=5)

In [ ]:
# Use best model to predict label

# load weight
model = createModel(best_lr, best_reg)
model.load_weights('./weight/myBestweight')

train_loss, train_acc = model.evaluate(train_images, train_labels)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Train accuracy:', train_acc)
print('Test accuracy:', test_acc)

# predict
predictions = model.predict(test_images)
print predictions[0]
print np.argmax(predictions[0])
print test_labels[0]

In [ ]:
# Initialize tensorboard
tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=0,
                          write_graph=True, write_images=False)

In [ ]:
# Use the best model to train longer (marathon)
# Load weight, lr, reg
# Add checkpoint & tensorboard metrics

def marathon(lr,reg,epoch,batch_size,tensorboard):
    # Timing
    tic = time.time()
        
    # model
    model = createModel(lr,reg)
    model.load_weights('./weight/myBestweight')
    #model.summary()
        
    # training
    model.fit(train_images, train_labels,
              validation_data = (test_images,test_labels),
              epochs=epoch,
              batch_size=batch_size,
              shuffle=True,
              callbacks=[tensorboard])
        
    # evaluate result
    train_loss, train_acc = model.evaluate(train_images, train_labels)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print('Train accuracy:', train_acc)
    print('Test accuracy:', test_acc)
        

    model.save_weights('./weight/marathonWeight')
    
    toc = time.time()
    print 'Total Training: computed in %fs' % (toc - tic)

In [ ]:
marathon(best_lr,best_reg,epoch=1000,batch_size=10,tensorboard=tensorboard)